In [32]:
# importing dependencies
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# For model building
from sklearn.model_selection import train_test_split

# Models
from sklearn.ensemble import RandomForestRegressor

# Optimizations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## This notebook will be used for building a random forest in python.

Loading in the data to be used

In [2]:
# Load in the data
weather = pd.read_csv("~/Projects/NiceRide/Weather_data/01012010_12312017.csv") # Weather data

temp = []
for x in [2010 + x for x in range(8)] :
#   add dtype declaration to remove warning
    x = pd.read_csv("~/Projects/NiceRide/Nice_Ride_data/"+str(x)+"/NiceRide_trip_history_"+str(x)+".csv")
                   # dtype = {"Start station ID" : pd.int64, "End station ID" : pd.int64, "Account type": pd.Categorical}
    temp.append(x)
    nr = pd.concat(temp)

/home/grimoire/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Changing dates from objects into datetimes
weather['DATE'] = weather['DATE'].apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d'))

nr['Start date'] = nr['Start date'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %H:%M'))
nr['End date'] = nr['End date'].apply(lambda x: dt.datetime.strptime(x,'%m/%d/%Y %H:%M'))

Correcting missing data in the weather dataframe

In [4]:
# Changing TAVG-NAN values to the mean of the TMAX and TMIN
weather = weather.fillna(value={'TAVG': weather[weather.TAVG.isna()][['TMAX', 'TMIN']]
                                .agg("mean", axis="columns")})

### Preparing the data to be used for the model

In [5]:
# Resampling our NR data so it takes the start date and Total duration, returns a count of rides per day 'daily_totals'

daily_totals = nr[['Start date', 'Total duration (seconds)']].resample('D', on='Start date').count()
daily_totals = daily_totals.drop('Start date', axis=1)
daily_totals = daily_totals.reset_index()
daily_totals = daily_totals.rename(index=str, columns={"Total duration (seconds)": "DCOUNT",'Start date':'DATE'})

In [6]:
print(daily_totals.head())
print(daily_totals.describe())

        DATE  DCOUNT
0 2010-06-07       1
1 2010-06-08       2
2 2010-06-09       7
3 2010-06-10     104
4 2010-06-11     287
            DCOUNT
count  2709.000000
mean    994.531931
std    1058.519993
min       0.000000
25%       0.000000
50%     722.000000
75%    1732.000000
max    5625.000000


The daily totals DataFrame looks good

In [7]:
print(weather.keys())
# The weather station and name won't be necessary or add anything to our analysis
weather = weather.drop(['STATION', 'NAME'], axis=1)

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG',
       'TMAX', 'TMIN'],
      dtype='object')


In [8]:
weather.head()

,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
0,2010-01-01,8.95,0.0,0.0,9,-1.5,6,-9
1,2010-01-02,4.03,0.0,0.0,9,-7.0,1,-15
2,2010-01-03,2.91,0.0,0.0,9,-3.5,7,-14
3,2010-01-04,5.59,0.0,0.0,9,-1.5,7,-10
4,2010-01-05,3.13,0.0,0.0,9,0.5,10,-9


In [9]:
temp = weather.merge(daily_totals, how='outer',on='DATE')
temp.info()
temp.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2922 entries, 0 to 2921
Data columns (total 9 columns):
DATE      2922 non-null datetime64[ns]
AWND      2922 non-null float64
PRCP      2922 non-null float64
SNOW      2922 non-null float64
SNWD      2922 non-null int64
TAVG      2922 non-null float64
TMAX      2922 non-null int64
TMIN      2922 non-null int64
DCOUNT    2709 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 228.3 KB


,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DCOUNT
0,2010-01-01,8.95,0.0,0.0,9,-1.5,6,-9,NaN
1,2010-01-02,4.03,0.0,0.0,9,-7.0,1,-15,NaN
2,2010-01-03,2.91,0.0,0.0,9,-3.5,7,-14,NaN
3,2010-01-04,5.59,0.0,0.0,9,-1.5,7,-10,NaN
4,2010-01-05,3.13,0.0,0.0,9,0.5,10,-9,NaN


In [10]:
# Notice how the count of DCOUNT varies from what the other columns have
features = temp.fillna(value=0,axis=0) # Where we don't have a daily count (Dcount) fill this with zeros instead
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2922 entries, 0 to 2921
Data columns (total 9 columns):
DATE      2922 non-null datetime64[ns]
AWND      2922 non-null float64
PRCP      2922 non-null float64
SNOW      2922 non-null float64
SNWD      2922 non-null int64
TAVG      2922 non-null float64
TMAX      2922 non-null int64
TMIN      2922 non-null int64
DCOUNT    2922 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 228.3 KB


In [11]:
features.head()

,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DCOUNT
0,2010-01-01,8.95,0.0,0.0,9,-1.5,6,-9,0.0
1,2010-01-02,4.03,0.0,0.0,9,-7.0,1,-15,0.0
2,2010-01-03,2.91,0.0,0.0,9,-3.5,7,-14,0.0
3,2010-01-04,5.59,0.0,0.0,9,-1.5,7,-10,0.0
4,2010-01-05,3.13,0.0,0.0,9,0.5,10,-9,0.0


This next section will change our date column from dtype datetime into seperate numerical relevant date colummns

In [12]:
features['YEAR'] = features['DATE'].dt.year
features['MONTH'] = features['DATE'].dt.month
features['DAY'] = features['DATE'].dt.day

In [13]:
features.head() # Three new, seperate, columns that identify year, month, day

,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DCOUNT,YEAR,MONTH,DAY
0,2010-01-01,8.95,0.0,0.0,9,-1.5,6,-9,0.0,2010,1,1
1,2010-01-02,4.03,0.0,0.0,9,-7.0,1,-15,0.0,2010,1,2
2,2010-01-03,2.91,0.0,0.0,9,-3.5,7,-14,0.0,2010,1,3
3,2010-01-04,5.59,0.0,0.0,9,-1.5,7,-10,0.0,2010,1,4
4,2010-01-05,3.13,0.0,0.0,9,0.5,10,-9,0.0,2010,1,5


In [14]:
# Random forests can't use datetime objects
features = features.drop('DATE', axis=1)

In [15]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2922 entries, 0 to 2921
Data columns (total 11 columns):
AWND      2922 non-null float64
PRCP      2922 non-null float64
SNOW      2922 non-null float64
SNWD      2922 non-null int64
TAVG      2922 non-null float64
TMAX      2922 non-null int64
TMIN      2922 non-null int64
DCOUNT    2922 non-null float64
YEAR      2922 non-null int64
MONTH     2922 non-null int64
DAY       2922 non-null int64
dtypes: float64(5), int64(6)
memory usage: 273.9 KB


In [16]:
# Need to drop all days where there where no rides taken (Dcount = 0)
features = features[features['DCOUNT'] != 0].reset_index(drop=True)

In [17]:
features.describe()

,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DCOUNT,YEAR,MONTH,DAY
count,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000,1667.000000
mean,8.857259,0.126023,0.015357,0.064787,63.063287,72.171566,53.988002,1616.188962,2013.628674,7.191962,15.769646
std,3.451232,0.344777,0.224634,0.582355,13.021535,13.991099,12.666982,903.294550,2.230847,2.051933,8.854145
min,0.890000,0.000000,0.000000,0.000000,8.000000,19.000000,-3.000000,1.000000,2010.000000,1.000000,1.000000
25%,6.490000,0.000000,0.000000,0.000000,54.000000,63.000000,45.000000,883.000000,2012.000000,6.000000,8.000000
50%,8.500000,0.000000,0.000000,0.000000,66.000000,75.000000,56.000000,1501.000000,2014.000000,7.000000,16.000000
75%,10.960000,0.050000,0.000000,0.000000,73.000000,83.000000,64.000000,2291.500000,2016.000000,9.000000,23.000000
max,23.710000,4.130000,6.400000,8.000000,91.000000,103.000000,81.000000,5625.000000,2017.000000,11.000000,31.000000


DCOUNT shows Minimum of at least 1, and the

I'm concerned why month shows min 1; there shouldn't be any data for January still. Time to investigate further

In [18]:
features[features.MONTH == 1]

,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DCOUNT,YEAR,MONTH,DAY
585,7.61,0.00,0.0,3,25.5,35,16,1.0,2013,1,8
586,10.29,0.28,0.0,2,31.5,39,24,1.0,2013,1,10


In [19]:
type(features.MONTH)

pandas.core.series.Series

Data looks good!

### Features, Targets, and converting Data in Arrays (optimizes performance)

In [20]:
# Saving the target/label variable and removing from features Dataframe
target = np.array(features.DCOUNT)
feat = features.drop('DCOUNT', axis=1)

# Saving the feature names for future use
feature_names = list(features.columns)

feat = np.array(feat)

In [21]:
# Splitting the data into training and test sets
feat_train, feat_test, label_train, label_test = train_test_split(feat, target, test_size=.25, random_state=42)

Inspecting the shape of the features and labels

In [22]:
print('Training Features shape', feat_train.shape)
print('Training labels shape', label_train.shape)

print('Testing Features shape', feat_test.shape)
print('Testing labels shape', label_test.shape)

Training Features shape (1250, 11)
Training labels shape (1250,)
Testing Features shape (417, 11)
Testing labels shape (417,)


## Establishing a baseline

Need to find what a good baseline would be

In [23]:
'''
Insert baseline creation code here
'''

'\nInsert baseline creation code here\n'

## Training the data

In [24]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators=1000, random_state=42)

rf.fit(feat_train, label_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

## Making predictions on the test set

In [26]:
predictions = rf.predict(feat_test)

Mean Absolute Error:  2.14


## Assessing the performance of our RandomForestRegressor

In [31]:
errors = abs(predictions - label_test)

print('Mean Absolute Error: ', round(np.mean(errors), 2))

Mean Absolute Error:  2.14


In [27]:
# Calculating the Mean Absolute Percentage Error (MAPE)
mape = 100 * (errors / label_test)

accuracy = 100 - np.mean(mape)
print('Accuracy: ', np.round(accuracy, 2), '%')

Accuracy:  98.04 %


In [34]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_names, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: DCOUNT               Importance: 1.0
Variable: AWND                 Importance: 0.0
Variable: PRCP                 Importance: 0.0
Variable: SNOW                 Importance: 0.0
Variable: SNWD                 Importance: 0.0
Variable: TAVG                 Importance: 0.0
Variable: TMAX                 Importance: 0.0
Variable: TMIN                 Importance: 0.0
Variable: YEAR                 Importance: 0.0
Variable: MONTH                Importance: 0.0
Variable: DAY                  Importance: 0.0


In [ ]:
%matplotlib inline

# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [30]:
import matplotlib.pyplot as plt

# overall accuracy
acc = rf.score(feat_test,label_test)

# get roc/auc info
Y_score = rf.predict_proba(feat_test)[:,1]
fpr = dict()
tpr = dict()
fpr, tpr, _ = roc_curve(label_test, Y_score)

roc_auc = dict()
roc_auc = auc(fpr, tpr)

# make the plot
plt.figure(figsize=(10,10))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(True)
plt.plot(fpr, tpr, label='AUC = {0}'.format(roc_auc))        
plt.legend(loc="lower right", shadow=True, fancybox =True) 
plt.show()

AttributeError: 'RandomForestRegressor' object has no attribute 'predict_proba'